![](https://raw.githubusercontent.com/Databricks-BR/workshop_agents/refs/heads/main/demo-main/img/header_workshop.png)


 Item | Description |
 --- | --- |
 **Objective** | Create an Agent Supervisor with Agent Bricks |
 **Databricks Run Time** | DBR 16.4 LTS |
 **Language** | Python and Pyspark |


![](https://raw.githubusercontent.com/Databricks-BR/workshop_agents/refs/heads/main/demo-main/img/img/06_diagram.png)

## Create an Agent Supervisor with Agent Bricks

![](https://raw.githubusercontent.com/Databricks-BR/workshop_agents/refs/heads/main/demo-main/img/img/07_agent_bricks.png)

**Agent Bricks** is a streamlined platform designed to make deploying, managing, and scaling intelligent agents remarkably simple for every organization. With a focus on low-friction, user-friendly workflows, Agent Bricks empowers teams to quickly build, customize, and operationalize AI-powered agents tailored to diverse business needs—without the complexity of traditional software development. Its modular, flexible approach enables customers to prototype, iterate, and launch powerful automation or analytics agents with minimal overhead, accelerating innovation and reducing time-to-value. Whether you’re looking to enhance customer support, automate data workflows, or unlock new AI-driven insights, Agent Bricks provides an intuitive foundation for rapid experimentation and effortless deployment of trusted AI solutions.

In [0]:
#Read pdf file from /Volumes/vinicius_fialho_testes/workshop_ml_agentes/unstructured_data/meli-2025-08-04-letter.pdf and use ai_parse_document to extract
from pyspark.sql.functions import expr
df=spark.read.format("binaryFile").load("/Volumes/vinicius_fialho_testes/workshop_ml_agentes/unstructured_data/meli-2025-08-04-letter.pdf").withColumn(


    "parsed",
    expr("ai_parse_document(content)"))
display(df)

In [0]:
#Extract all columns from dataframe
from pyspark.sql.functions import col, parse_json

df_copy=df.withColumn(
   "parsed_json",
   parse_json(col("parsed").cast("string"))) \
 .selectExpr(
   "path",
   "parsed_json:document:elements")
display(df_copy)

In [0]:
#Explode into multiple rows with content from each page
from pyspark.sql.functions import explode, from_json
from pyspark.sql.types import ArrayType, StringType


# Define the expected array type for your data
array_schema = ArrayType(StringType())

# Convert 'elements' (VARIANT) to array by parsing as JSON string
df_copy2 = df_copy.withColumn("elements_array", from_json(col("elements").cast("string"), array_schema))

# Explode the new array column
df_copy3 = df_copy2.select("path", explode(col("elements_array")).alias("element"))
display(df_copy3)

In [0]:
display(df_copy3)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

#Add id to the dataframe
df_copy4=df_copy3.withColumn("id", monotonically_increasing_id())
display(df_copy4)

In [0]:
#Write dataframe to table vinicius_fialho_testes.workshop_ml_agentes.document_meli
df_copy4.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("vinicius_fialho_testes.workshop_ml_agentes.document_meli")

## Agente: 
https://e2-demo-field-eng.cloud.databricks.com/ml/bricks/mas/configure/b6af9a59-b66b-4d32-8011-8f9c2fa29d4d?o=1444828305810485

* ¿Que productos tienen un inventario inferior a 200 unidades?

* ¿Cual es la tasa de crecimiento de Brasil YoY en dolares en Q2 del 2025 para el revenue consolidado?





